# LEARNING PYTORCH WITH EXAMPLES
- 해당 튜토리얼은 다양한 예시를 통해 **파이토치** 의 핵심 개념을 소개함
>**파이토치의 두 가지 핵심 특징**
>1. numpy 의 array 와 흡사한 n-차원 **텐서**. 텐서는 GPU 상에서 동작할 수 있음
>2. 인공신경망을 설계하고 학습하기 위한 **Automatic differentitaion**

- **여기선 ```y = sin(x)``` 를 3차 다항식으로 fitting 하는 문제를 예시로 듬**
- 네트워크는 4개의 파라미터를 가지며 유클라디안 거리를 최소화하는 방식으로 gradient descent 를 통해 최적화할 것

---
# 1. Tensors

## 1.1 Warm-up: numpy
- 파이토치로 바로 구현하기 앞서 numpy 배열을 이용한 구현을 먼저 해보겠음
- numpy 는 연산 그래프나 딥 러닝에 대한 어떤 것도 적용되어 있지 않지만 그래도 해당 문제에 적용은 가능

In [10]:
import numpy as np
import math

x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    y_pred = a + b*x + c*x**2 + d*x**3
    loss = np.square(y_pred - y).sum()
    
    if t % 100 == 99:
        print(t, loss)
    
    grad_y_pred = 2.0 * (y_pred - y)
    
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x**2).sum()
    grad_d = (grad_y_pred * x**3).sum()
    
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f"Result: y = {a} + {b}x + {c}x^2 + {d}x^3")

99 1813.5364278327934
199 1273.72760597244
299 895.866082742301
399 631.2046172607264
499 445.72355660426643
599 315.6624659912412
699 224.41461776209417
799 160.3653631524902
899 115.38625175130733
999 83.7851296711114
1099 61.57358073031236
1199 45.95541081337591
1299 34.96922284307594
1399 27.238492906400417
1499 21.79670056661665
1599 17.964897471379363
1699 15.265938384639387
1799 13.364361781069498
1899 12.02422655460381
1999 11.079526565073337
Result: y = -0.04920264078751947 + 0.8469417881331966x + 0.008488274271323823x^2 + -0.09193655880622141x^3


---
# 1.2 PyTorch: Tensors
- Numpy 는 매우 훌륭한 프레임워크지만 GPU 를 지원하지 않음
- 최신 심층신경망은 GPU 환경에서 수배~수십배 빠른 속도를 보이므로 numpy 는 딥러닝에 적합하지 않음
- 그래서 PyTorch 는 **텐서** 를 사용함. 개념상 numpy array 와 동일하지만 인공신경망 학습을 위한 연산 추적 및 다양한 연산 함수를 제공함
- 또한 **텐서를 GPU 상에 올려 동작하는 것이 가능**함

- 위 예시를 torch 를 이용하여 구현해보겠음

In [12]:
import torch
import math

dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    y_pred = a + b*x + c*x**2 + d*x**3
    loss = (y_pred - y).pow(2).sum().item()
    
    if t%100 == 99:
        print(t, loss)
        
    grad_y_pred = 2.0 * (y_pred - y)
    
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x**2).sum()
    grad_d = (grad_y_pred * x**3).sum()
    
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f"Result: y = {a.item()} + {b.item()}x + {c.item()}x^2 + {d.item()}x^3")

99 1499.564208984375
199 1004.4855346679688
299 674.2183837890625
399 453.7781982421875
499 306.5608825683594
599 208.18621826171875
699 142.4093017578125
799 98.3998031616211
899 68.9347915649414
999 49.19337463378906
1099 35.95713806152344
1199 27.07569122314453
1299 21.111589431762695
1399 17.10318374633789
1499 14.406929016113281
1599 12.5916748046875
1699 11.368440628051758
1799 10.543407440185547
1899 9.986397743225098
1999 9.609949111938477
Result: y = -0.018315279856324196 + 0.8351486921310425x + 0.0031596915796399117x^2 + -0.09025909006595612x^3


---
# 2. Autograd

## 2.1 PyTorch: Tensors and autograd
- 위 예씨에선 **forward, backward** 과정을 직접 작성하여 적용함
- 직접 구현하는 작업은 네트워크가 작으면 간단하지만 모델이 커질수록 감당하기 어려워짐
- 다행히 파이토치가 제공하는 **autograd** 패키지를 통해 backward 연산에서 필요한 자동 미분을 이용할 수 있음
- autograd 를 사용하면 네트워크가 **computational graph**를 정의함. 그래프의 노드는 텐서가 되고 엣지는 출력 텐서를 만드는 연산이 됨. 이 그래프에 대한 역전파로 gradient 를 계산하게 됨

- 설명으론 복잡하지만 실제로 사용하기는 간단함
- 그래프를 이루는 텐서의 ```requires_grad=True``` 특성을 설정하면 ```.grad``` 특성에 해당 텐서가 사용된 연산에 대한 gradient 를 축적함

- 이제 위 문제를 autograd 를 이용하여 구현해 보겠음
- 더이상 backward 연산을 직접 구현하지 않아도 됨

In [16]:
import torch
import math

dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    y_pred = a + b*x + c*x**2 + d*x**3
    loss = (y_pred - y).pow(2).sum()
    
    if t%100 == 99:
        print(t, loss.item())
        
    loss.backward()
    
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad
        
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f"Result: y = {a.item()} + {b.item()}x + {c.item()}x^2 + {d.item()}x^3")

99 4690.0546875
199 3130.77197265625
299 2091.953125
399 1399.5548095703125
499 937.8284912109375
599 629.7680053710938
699 424.12115478515625
799 286.7630615234375
899 194.96263122558594
999 133.571533203125
1099 92.48971557617188
1199 64.97978210449219
1299 46.545005798339844
1399 34.182437896728516
1499 25.88555335998535
1599 20.312847137451172
1699 16.566728591918945
1799 14.046306610107422
1899 12.349081039428711
1999 11.20508861541748
Result: y = 0.029936792328953743 + 0.818003237247467x + -0.0051645925268530846x^2 + -0.08782030642032623x^3


## 2.2 PyTorch: Defining new autograd functions
>**autograd operator 는 두 가지 함수를 텐서에 적용함**
>1. **forward** : 입력 텐서로부터 출력 텐서를 계산
>2. **backward** : 출력 텐서의 gradient 를 입력받아 입력 텐서에 대한 gradient 를 계산

- 파이토치에선 ```torch.autograd.Function``` 클래스를 상속하여 **사용자 정의 autograd function** 을 만들 수 있음
- 해당 클래스에 ```forward, backward``` 메서드를 구현하면 됨
- 클래스의 인스턴스를 만들고 이를 텐서를 입력받는 함수로 호출하는 것으로 사용 가능

- 예시로 **Legendre polynomial** 다항식을 구현해 보겠음

In [17]:
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input_data):
        ctx.save_for_backward(input_data)
        return 0.5 * (5*input_data**3 - 3*input_data)
    
    @staticmethod
    def backward(ctx, grad_output):
        input_data, = ctx.saved_tensors
        return grad_output * 1.5 * (5 *input_data**2 - 1)

In [19]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    P3 = LegendrePolynomial3.apply
    y_pred = a + b*P3(c + d*x)
    
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    loss.backward()
    
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad
        
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None
        
print(f"Result: y = {a.item()} + {b.item()}*P3({c.item()} + {d.item()}x)")

99 209.95834350585938
199 144.66018676757812
299 100.70249938964844
399 71.03520202636719
499 50.978515625
599 37.40313720703125
699 28.20686912536621
799 21.973186492919922
899 17.745729446411133
999 14.877889633178711
1099 12.931766510009766
1199 11.610918998718262
1299 10.714248657226562
1399 10.105474472045898
1499 9.692106246948242
1599 9.411375045776367
1699 9.220745086669922
1799 9.091285705566406
1899 9.003361701965332
1999 8.943639755249023
Result: y = -1.765793067320942e-11 + -2.208526849746704*P3(9.924167737596079e-11 + 0.2554861009120941x)


---
# 3. ```nn.module```

## 3.1 PyTorch: nn
- computational graph 와 autograd 는 복잡한 연산의 정의와 derivatives 자동 계산에 매우 강력함
- 하지만 매우 큰 네트워크에 대해서 raw autograd 는 너무 low-level 임
- 네트워크를 구축할 때 우리는 **학습 가능한 파라미터로 이루어진 레이어에 연산을 구현함**
- TensorFlow, Keras, TFLearn 같은 패키지는 네트워크를 간편하게 구축할 수 있도록 다양한 high-level 레이어를 제공함

- 파이토치에선 ```nn``` 패키지가 그 역할을 해줌
- ```nn``` 패키지는 네트워크의 레이어와 동일한 역할을 하는 **Module** 을 정의함
- 모듈은 입력 텐서를 받아 출력 텐서를 계산함.
- ```nn``` 패키지는 또한 loss function 도 정의할 수 있음

- 위 예시에서 사용한 다항 모델을 ```nn``` 패키지를 이용하여 구현해 보겠음

In [5]:
import torch
import math

x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

loss_fn = torch.nn.MSELoss(reduction="sum")

learning_rate = 1e-6
for t in range(2000):
    y_pred = model(xx)
    loss = loss_fn(y_pred, y)
    
    if t % 100 == 99:
        print(t, loss.item())
        
    model.zero_grad()
    loss.backward()
    
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
    
linear_layer = model[0]

print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + \
      {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 853.07666015625
199 567.9133911132812
299 379.1068420410156
399 254.08680725097656
499 171.29574584960938
599 116.46434020996094
699 80.14610290527344
799 56.087615966796875
899 40.148502349853516
999 29.58719825744629
1099 22.588254928588867
1199 17.94947052001953
1299 14.874412536621094
1399 12.835579872131348
1499 11.483595848083496
1599 10.586894035339355
1699 9.992032051086426
1799 9.597319602966309
1899 9.33536148071289
1999 9.161462783813477
Result: y = 0.005298037081956863 + 0.8393722176551819 x +       -0.0009139998001046479 x^2 + -0.09085985273122787 x^3


## 3.2 PyTorch: optim
- 모델의 파라미터를 업데이트할 때 ```torch.no_grad``` 블록 상에서 일일히 진행했음
- 이는 stochastic gradient descent 에선 어렵지 않으나 더 상위 알고리즘에선 직접 구현하기가 매우 어려움

- 파이토치는 ```optim``` 패키지를 제공하여 파라미터의 최적화를 간단하게 해줌

In [7]:
import torch
import math

x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

loss_fn = torch.nn.MSELoss(reduction="sum")

learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for t in range(2000):
    y_pred = model(xx)
    loss = loss_fn(y_pred, y)
    
    if t % 100 == 99:
        print(t, loss.item())
        
    model.zero_grad()
    loss.backward()
    
    optimizer.step()
    
linear_layer = model[0]

print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + \
      {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 890.3433227539062
199 600.4813232421875
299 494.2259826660156
399 386.91204833984375
499 282.89788818359375
599 194.0765380859375
699 125.3480224609375
799 76.06620788574219
899 43.026580810546875
999 22.971927642822266
1099 12.999251365661621
1199 9.520439147949219
1299 8.8980712890625
1399 8.864032745361328
1499 8.880899429321289
1599 8.939823150634766
1699 8.940942764282227
1799 8.913397789001465
1899 8.913410186767578
1999 8.9237699508667
Result: y = 0.0005278979078866541 + 0.8562362790107727 x +       0.0005281136254779994 x^2 + -0.09383614361286163 x^3


## 3.3 PyTorch: Custom nn Modules
- 이미 존재하는 모델보다 더 복잡한 모델을 구현하고 싶다면 **사용자 정의 모델**을 구축해야 함
- ```nn.Module``` 클래스를 상속하여 사용자 정의 모델을 구축할 수 있음

In [8]:
import torch
import math

class Polynomial3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        
    def forward(self, x):
        return self.a + self.b*x + self.c*x**2 + self.d*x**3
    
    def string(self):
        return f"y = {self.a.item()} + {self.b.item()}x + {self.c.item()}x^2 + {self.d.item()}x^3"

In [9]:
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

model = Polynomial3()

criterion = torch.nn.MSELoss(reduction="sum")
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

for t in range(2000):
    y_pred = model(x)
    
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f"Result: {model.string()}")

99 1168.9569091796875
199 792.3114624023438
299 538.475341796875
399 367.2430725097656
499 251.62144470214844
599 173.47265625
699 120.59778594970703
799 84.78569030761719
899 60.50464630126953
999 44.023948669433594
1099 32.82545852661133
1199 25.20763397216797
1299 20.019813537597656
1399 16.48282241821289
1499 14.06859016418457
1599 12.418822288513184
1699 11.290163040161133
1799 10.517105102539062
1899 9.987018585205078
1999 9.623120307922363
Result: y = 0.023523803800344467 + 0.8395689129829407x + -0.004058246500790119x^2 + -0.09088782966136932x^3
